In [1]:
import pandas as pd
import numpy as np
import random
import re
import xlwt

In [2]:
# read excel
df_test = pd.read_excel("random_data.xlsx")
df = df_test.copy()
df

,Unnamed: 0,序号,提交答卷时间,所用时间,来源,来源详情,来自IP,1、姓名,2、微信号（请务必确保准确、可添加。这是小助手联系您参加活动的方式）,3、请填写你的学校邮箱（例：hop1@jhu.edu）,...,24、(武侠),24、(动画),24、(战争),24、(悬疑),24、(恐怖),24、(真人秀),24、(其它),25、您期待的恋爱相处模式？,26、您认为最浪漫的事？,27、若未为您成功匹配到CP，您是否愿意与并非是您选择的性别（比如同性）的参与者一对一地交友？
0,0,1,test1,test1,test1,test1,test1,test1,test1,test1,...,1,0,0,0,0,1,0,empty,empty,2
1,1,2,test2,test2,test2,test2,test2,test2,test2,test2,...,1,0,1,1,1,0,1,empty,empty,2
2,2,3,test3,test3,test3,test3,test3,test3,test3,test3,...,1,1,0,0,1,1,1,empty,empty,1
3,3,4,test4,test4,test4,test4,test4,test4,test4,test4,...,0,1,1,1,0,1,1,empty,empty,2
4,4,5,test5,test5,test5,test5,test5,test5,test5,test5,...,1,0,1,0,0,1,1,empty,empty,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,496,test496,test496,test496,test496,test496,test496,test496,test496,...,1,1,1,1,0,0,0,empty,empty,1
496,496,497,test497,test497,test497,test497,test497,test497,test497,test497,...,0,0,0,1,1,1,1,empty,empty,2
497,497,498,test498,test498,test498,test498,test498,test498,test498,test498,...,0,0,1,1,0,1,0,empty,empty,2
498,498,499,test499,test499,test499,test499,test499,test499,test499,test499,...,0,0,1,1,1,0,0,empty,empty,2


In [3]:
# 查重(机器没法防止对方写错号码刚好一样的情况，所以如果不匹配，建议人工检查)
# 保留微信号最后一列重复
df.drop_duplicates(subset=['2、微信号（请务必确保准确、可添加。这是小助手联系您参加活动的方式）'], keep='last', inplace=True)
# 保留邮箱最后一列重复
df.drop_duplicates(subset=['3、请填写你的学校邮箱（例：hop1@jhu.edu）'], keep='last', inplace=True)
len(df) == len(df_test)

True

In [4]:
# save q,实现方法是获取相同题目的index对，例如{1:[0,0]}题目1的index从0-1开始
df1 = df.copy()
df1 = df1.iloc[:,7:]
index_list = [i.split("、")[0] for i in df1.columns.tolist()]
index_record_dict = {}

In [5]:
for i in range(1,28):
    meet = 0
    for j in range(len(index_list)):
        if index_list[j] == str(i) and meet == 0:
            start_index = j
            meet += 1
            if j == len(index_list) - 1:
                final_index = j
            elif index_list[j+1] != i:
                final_index = j
        if index_list[j] == str(i) and meet == 1:
            final_index = j
    index_record_dict[str(i)] = [start_index,final_index]
print(index_record_dict)

{'1': [0, 0], '2': [1, 1], '3': [2, 2], '4': [3, 3], '5': [4, 6], '6': [7, 7], '7': [8, 24], '8': [25, 25], '9': [26, 26], '10': [27, 27], '11': [28, 28], '12': [29, 34], '13': [35, 35], '14': [36, 36], '15': [37, 37], '16': [38, 40], '17': [41, 41], '18': [42, 44], '19': [45, 45], '20': [46, 46], '21': [47, 47], '22': [48, 55], '23': [56, 67], '24': [68, 81], '25': [82, 82], '26': [83, 83], '27': [84, 84]}


In [6]:
list(index_record_dict.keys())[0]

'1'

In [7]:
# 根据index对获取df1信息
def ini_dict():
    df1_dict = {}
    for idx,data in df1.iterrows():
        node_dict = {}
        for key in list(index_record_dict.keys()):
            index_list_get = index_record_dict[key]
            if index_list_get[0] == index_list_get[1]:
                node_dict[str(key)] = data[index_list_get[0]]
            else:
                node_dict[str(key)] = data[index_list_get[0]:index_list_get[1]+1].tolist()
        node_dict["100"] = []
        df1_dict[idx] = node_dict
    # add 100
    for key in list(df1_dict.keys()):
        df1_dict[key]["100"] = [list(df1_dict.keys())]
    #复制结果
    df1_dict_2 = df1_dict.copy()
    #看看结果
    #print(df1_dict_2[0])
    return df1_dict_2

In [8]:
# 筛选
require_man_id = []

In [9]:
# gender filter func
def gender_filter(gd_list):
    tmp_list = []
    for i in range(len(gd_list)):
        if gd_list[i] == 1:
            tmp_list.append(i+1)
    return tmp_list

In [10]:
# compare two list 交集 func
def get_interaction(dict1):
    for key in list(dict1.keys()):
        if len(dict1[key]["100"]) == 1:
            return dict1
        list_1 = dict1[key]["100"][0]
        list_2 = dict1[key]["100"][1]
        list_3 = list(set(list_1).intersection(set(list_2)))
        dict1[key]["100"] = [list_3]
    return dict1

In [11]:
# 筛选1-性别
def sel_gender(df1_dict_2):
    df1_dict_gender = df1_dict_2.copy()
    #选择合适的对象
    for key in list(df1_dict_gender.keys()):
        req_list = gender_filter(df1_dict_gender[key]["5"])
        tmp_list = []
        for key2 in list(df1_dict_gender.keys()):
            if key != key2:
                if df1_dict_gender[key2]["4"] in req_list:
                    tmp_list.append(key2)
        df1_dict_gender[key]["100"].append(tmp_list)
    # 取性别和原始交集
    df1_dict_gender = get_interaction(df1_dict_gender)
    print(len(df1_dict_gender[0]["100"][0]))
    return df1_dict_gender

In [12]:
#筛选2-年龄
def sel_old(df1_dict_gender):
    df1_dict_old = df1_dict_gender.copy()
    for key in list(df1_dict_old.keys()):
        req_list = gender_filter(df1_dict_old[key]["7"])
        tmp_list = []
        for key2 in list(df1_dict_old.keys()):
            if key != key2:
                if df1_dict_old[key2]["6"] in req_list:
                    tmp_list.append(key2)
        df1_dict_old[key]["100"].append(tmp_list)
    #取性别和年龄交集
    df_1_dict_gender_old = get_interaction(df1_dict_old)
    print(len(df_1_dict_gender_old[0]["100"][0]))
    return df_1_dict_gender_old

In [13]:
#筛选3-学校
def sel_school(df_1_dict_gender_old):
    df1_dict_school = df_1_dict_gender_old.copy()
    for key in list(df1_dict_school.keys()):
        req_school = df1_dict_school[key]["9"]
        tmp_list = []
        for key2 in list(df1_dict_school.keys()):
            if key != key2:
                if req_school == 1 and df1_dict_school[key]["8"] == df1_dict_school[key2]["8"]:
                    tmp_list.append(key2)
                elif req_school == 2 and df1_dict_school[key]["8"] != df1_dict_school[key2]["8"]:
                    tmp_list.append(key2)
                elif req_school == 3:
                    tmp_list.append(key2)
        df1_dict_school[key]["100"].append(tmp_list)

    #取学校和性别、年龄交集
    df_1_dict_school_gender_old = get_interaction(df_1_dict_gender_old)
    print(len(df_1_dict_school_gender_old[0]["100"][0]))
    return df_1_dict_school_gender_old

In [14]:
#筛选4-恋爱目的
def sel_goal(df_1_dict_school_gender_old):
    df1_dict_school = df_1_dict_school_gender_old.copy()
    for key in list(df1_dict_school.keys()):
        tmp_list = []
        for key2 in list(df1_dict_school.keys()):
            if key != key2:
                if df1_dict_school[key]["10"] == df1_dict_school[key2]["10"]:
                    tmp_list.append(key2)
        df1_dict_school[key]["100"].append(tmp_list)

    #取学校和性别、年龄交集
    df_1_dict_goal_school_gender_old = get_interaction(df_1_dict_school_gender_old)
    print(len(df_1_dict_goal_school_gender_old[0]["100"][0]))
    return df_1_dict_goal_school_gender_old

In [15]:
#筛选5-时间
def sel_time(df_1_dict_goal_school_gender_old):
    df1_dict_school = df_1_dict_goal_school_gender_old.copy()
    for key in list(df1_dict_school.keys()):
        tmp_list = []
        for key2 in list(df1_dict_school.keys()):
            if key != key2:
                time_list_1 = df1_dict_school[key]["12"]
                time_list_2 = df1_dict_school[key2]["12"]
                for i in range(len(time_list_2)):
                    if time_list_1[i] == time_list_2[i] and time_list_1[i] == 1:
                        tmp_list.append(key2)
        df1_dict_school[key]["100"].append(tmp_list)

    #取学校和性别、年龄交集
    df_1_dict_time_goal_school_gender_old = get_interaction(df_1_dict_goal_school_gender_old)
    print(len(df_1_dict_time_goal_school_gender_old[0]["100"][0]))
    return df_1_dict_time_goal_school_gender_old

In [16]:
# 刚性select
df1_dict_2 = ini_dict()
df1_dict_gender = sel_gender(df1_dict_2)
df_1_dict_gender_old = sel_old(df1_dict_gender)
df_1_dict_school_gender_old = sel_school(df_1_dict_gender_old)
df_1_dict_goal_school_gender_old = sel_goal(df_1_dict_school_gender_old)
df_1_dict_time_goal_school_gender_old = sel_goal(df_1_dict_goal_school_gender_old)

323
164
44
17
17


In [17]:
# 查看
ini_dict()[0]

{'1': 'test1',
 '2': 'test1',
 '3': 'test1',
 '4': 2,
 '5': [0, 1, 1],
 '6': 13,
 '7': [0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 '8': 4,
 '9': 1,
 '10': 1,
 '11': 2,
 '12': [1, 0, 0, 1, 1, 0],
 '13': 2,
 '14': 3,
 '15': 2,
 '16': [1, 0, 1],
 '17': 1,
 '18': [1, 0, 1],
 '19': 10,
 '20': 3,
 '21': 5,
 '22': [1, 1, 0, 0, 0, 0, 1, 1],
 '23': [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 '24': [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0],
 '25': 'empty',
 '26': 'empty',
 '27': 2,
 '100': [[0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
  

In [18]:
# sel_a软性权重-单选：11,21
# sel_b软性权重-多选：22,23,24,
# sel_c软性权重-双题单选：(13，14)，(19,20)
# sel_d软性权重-双体单多选：(15,16)，(17,18)

In [19]:
# 软性权重-单选：11,21
def sel_a(df1_dict_3,item_num):
    item_num = str(item_num)
    new_dict = {}
    for key in list(df1_dict_3.keys()): 
        new_dict[key] = []
        for key2 in list(df1_dict_3.keys()): 
            if df1_dict_3[key][item_num] == df1_dict_3[key2][item_num]:
                val = 1
            else:
                val = 0
            new_dict[key].append(val)
    return new_dict

In [20]:
from sklearn.metrics.pairwise import cosine_similarity
def compute_corr(matrix):
    return cosine_similarity(matrix)

In [21]:
# 软性权重-多选：22,23,24,
def sel_b(df1_dict_3,item_num):
    item_num = str(item_num)
    new_dict = {}
    matrix = []
    for key2 in list(df1_dict_3.keys()): 
        matrix.append(df1_dict_3[key2][item_num])
    matrix_value = compute_corr(matrix)
    for key in list(df1_dict_3.keys()): 
        #new_dict[key] = [round(i,2) for i in list(matrix_value[int(key)])]
        new_dict[key] = list(matrix_value[int(key)])
    return new_dict

In [22]:
#  sel_c1314软性权重-双题单选：(13，14)
def sel_c1314(df1_dict_3,item_num13,item_num14):
    item_num13 = str(item_num13)
    item_num14 = str(item_num14)
    new_dict = {}
    for key in list(df1_dict_3.keys()): 
        new_dict[key] = []
        for key2 in list(df1_dict_3.keys()):
            val = 0
            if df1_dict_3[key][item_num14] == 3:
                val = 1
            elif df1_dict_3[key][item_num14] == 1 and df1_dict_3[key][item_num13] == df1_dict_3[key2][item_num13]:
                val = 1
            elif df1_dict_3[key][item_num14] == 2 and df1_dict_3[key][item_num13] != df1_dict_3[key2][item_num13]:
                val = 1
            else:
                val = 0
            new_dict[key].append(val)
    return new_dict

In [23]:
#  sel_c1920软性权重-双题单选：(19，20)
def sel_c1920(df1_dict_3,item_num19,item_num20):
    item_num19 = str(item_num19)
    item_num20 = str(item_num20)
    new_dict = {}
    for key in list(df1_dict_3.keys()): 
        new_dict[key] = []
        for key2 in list(df1_dict_3.keys()):
            val = 0
            if df1_dict_3[key][item_num20] == 3:
                val = 1
            elif df1_dict_3[key][item_num20] == 1 and df1_dict_3[key][item_num19] == df1_dict_3[key2][item_num19]:
                val = 1
            elif df1_dict_3[key][item_num20] == 2 and df1_dict_3[key][item_num19] != df1_dict_3[key2][item_num19]:
                val = 1
            else:
                val = 0
            new_dict[key].append(val)
    return new_dict

In [24]:
# sel_d软性权重-双体单多选：(15,16)，(17,18)
def sel_d(df1_dict_3,item_num15,item_num16):
    item_num15 = str(item_num15)
    item_num16 = str(item_num16)
    new_dict = {}
    for key in list(df1_dict_3.keys()): 
        new_dict[key] = []
        for key2 in list(df1_dict_3.keys()):
            val = 0
            get_list = gender_filter(df1_dict_3[key][item_num16])
            if df1_dict_3[key2][item_num15] in get_list:
                val = 1
            else:
                val = 0
            new_dict[key].append(val)
    return new_dict

In [25]:
### 汇总权重

In [26]:
# sel_a软性权重-单选：10,11,21
df1_dict_3 = ini_dict()
q_11 = sel_a(df1_dict_3,11)
q_21 = sel_a(df1_dict_3,21)

In [27]:
# sel_b软性权重-多选：22,23,24,
q_22 = sel_b(df1_dict_3,22)
q_23 = sel_b(df1_dict_3,23)
q_24 = sel_b(df1_dict_3,24)

In [28]:
#  sel_c1314软性权重-双题单选：(13，14)
q_1314 = sel_c1314(df1_dict_3,13,14)
#  sel_c1920软性权重-双题单选：(19，20)
q_1920 = sel_c1314(df1_dict_3,19,20)

In [29]:
# sel_d软性权重-双体单多选：(15,16)，(17,18)
q_1516 = sel_d(df1_dict_3,15,16)
q_1718 = sel_d(df1_dict_3,17,18)

In [30]:
# 汇总
all_list = [q_11,q_1314,q_1516,q_1718,q_1920,q_21,q_22,q_23,q_24]
l_weight = [1,1,1,1,1,1,1,1,1] #别改第11题权重

In [31]:
# 测试,需要全部相等
for i in all_list:
    print(len(i[0]),end = ",")

500,500,500,500,500,500,500,500,500,

In [32]:
### 计算总权重

In [33]:
weight_sum_dict = {}
weight_sum_dict = all_list[0]
for dict_index in range(len(all_list[1:])):
    dict_next = all_list[dict_index]
    for key in list(weight_sum_dict.keys()):
        weight_sum_dict[key] = [weight_sum_dict[key][i] + l_weight[dict_index]*dict_next[key][i] 
                                for i in range(len(weight_sum_dict[key]))]
for key in list(weight_sum_dict.keys()):
     weight_sum_dict[key] = [i/np.sum(l_weight) for i in weight_sum_dict[key]]

## 刚性和柔性合并

In [34]:
# 查看数据情况，序号0-499
df_1_dict_school_gender_old[0]

{'1': 'test1',
 '2': 'test1',
 '3': 'test1',
 '4': 2,
 '5': [0, 1, 1],
 '6': 13,
 '7': [0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1],
 '8': 4,
 '9': 1,
 '10': 1,
 '11': 2,
 '12': [1, 0, 0, 1, 1, 0],
 '13': 2,
 '14': 3,
 '15': 2,
 '16': [1, 0, 1],
 '17': 1,
 '18': [1, 0, 1],
 '19': 10,
 '20': 3,
 '21': 5,
 '22': [1, 1, 0, 0, 0, 0, 1, 1],
 '23': [1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 '24': [0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0],
 '25': 'empty',
 '26': 'empty',
 '27': 2,
 '100': [[352,
   1,
   322,
   99,
   90,
   360,
   73,
   266,
   42,
   491,
   270,
   220,
   20,
   149,
   121,
   314,
   156]]}

In [35]:
# 查看数据情况，序号0-499
weight_sum_dict[0]

[0.8888888888888888,
 0.6765839612209766,
 0.6431861260660904,
 0.5925865794675645,
 0.7120761311802843,
 0.6867126463821969,
 0.7162708188084932,
 0.8074561319166533,
 0.6745443711325345,
 0.6329914916731862,
 0.6963450208055423,
 0.4171058478532188,
 0.8103095836817296,
 0.7113675756193257,
 0.6431861260660904,
 0.5699934209276347,
 0.7345979329204854,
 0.5392837100659194,
 0.6047565601757848,
 0.5777065263464192,
 0.613733706901586,
 0.6648326865491095,
 0.5785674201318387,
 0.6671281918425855,
 0.4722222222222222,
 0.5406694893093822,
 0.7140832599187101,
 0.4626863596091464,
 0.5419923095092892,
 0.8168038011877453,
 0.4441414016839495,
 0.49471808596516414,
 0.8082281656333765,
 0.4471737627432871,
 0.48030069574328393,
 0.6867126463821969,
 0.6632321057816205,
 0.5465128803270584,
 0.542435233743133,
 0.42838465399598014,
 0.4188667065466112,
 0.602972148807599,
 0.3602570248938341,
 0.8339921051131616,
 0.5679197514961842,
 0.31814116477553467,
 0.7389284596687078,
 0.447173762

In [36]:
# 合并
final_new_dict = {}
for key in list(df_1_dict_school_gender_old.keys()):
    req_list = df_1_dict_school_gender_old[key]["100"][0]
    req_list.sort() #由于之前set乱序
    opt_list = weight_sum_dict[key]
    new_list = []
    for i in req_list:
        new_list.append([i,opt_list[i]])
    final_new_dict[key] = new_list

In [37]:
final_new_dict

{0: [[1, 0.6765839612209766],
  [20, 0.613733706901586],
  [42, 0.3602570248938341],
  [73, 0.6063451979096164],
  [90, 0.5218803805620751],
  [99, 0.5939093996674716],
  [121, 0.4829205326666604],
  [149, 0.5334185540103591],
  [156, 0.7136217347427913],
  [220, 0.4510651145773059],
  [266, 0.6311570908266413],
  [270, 0.4074074074074074],
  [314, 0.45303891879884733],
  [322, 0.5597987865347306],
  [352, 0.47412279858332007],
  [360, 0.5049256726613129],
  [491, 0.6812590187078946]],
 1: [[26, 0.35614920856386506],
  [37, 0.40995188860704995],
  [38, 0.7753602368953625],
  [50, 0.43535034497567343],
  [55, 0.7716994038041124],
  [57, 0.3814458557658022],
  [101, 0.309852792732757],
  [104, 0.3330559934947392],
  [139, 0.33907687980165363],
  [152, 0.4934996201030807],
  [161, 0.4209639038438681],
  [165, 0.43572836050280167],
  [167, 0.4719978423992433],
  [168, 0.41811297595778557],
  [187, 0.3579653151652258],
  [222, 0.3371691648694276],
  [238, 0.3371691648694276],
  [277, 0.3433

In [38]:
# Sort
final_out = {}
for pep in list(final_new_dict.keys()):
    pmatch_list = final_new_dict[pep]
    pmatch_sorted = sorted(pmatch_list, key = lambda pmatch_list:pmatch_list[1], reverse = True)
    final_out[pep] = pmatch_sorted

In [39]:
final_out

{0: [[156, 0.7136217347427913],
  [491, 0.6812590187078946],
  [1, 0.6765839612209766],
  [266, 0.6311570908266413],
  [20, 0.613733706901586],
  [73, 0.6063451979096164],
  [99, 0.5939093996674716],
  [322, 0.5597987865347306],
  [149, 0.5334185540103591],
  [90, 0.5218803805620751],
  [360, 0.5049256726613129],
  [121, 0.4829205326666604],
  [352, 0.47412279858332007],
  [314, 0.45303891879884733],
  [220, 0.4510651145773059],
  [270, 0.4074074074074074],
  [42, 0.3602570248938341]],
 1: [[38, 0.7753602368953625],
  [395, 0.7730118645762831],
  [55, 0.7716994038041124],
  [499, 0.6877469672094912],
  [296, 0.6109423262020128],
  [433, 0.5932847014689138],
  [405, 0.5740673244642684],
  [409, 0.5694444444444444],
  [327, 0.550758693263986],
  [152, 0.4934996201030807],
  [362, 0.47201896249328473],
  [167, 0.4719978423992433],
  [400, 0.4564089283421707],
  [475, 0.4487416816216459],
  [165, 0.43572836050280167],
  [50, 0.43535034497567343],
  [161, 0.4209639038438681],
  [168, 0.4181

In [40]:
# Save
name = []
match = []
score = []

for pep in list(final_out.keys()):
    for matches in final_out[pep]:
        name.append(pep)
        match.append(matches[0])
        score.append(matches[1])
        
out = pd.DataFrame()
out['Subject'] = name
out['Match'] = match
out['Score'] = score

out.to_excel('possiblematch.xlsx', encoding = 'utf-8', index = False)
